# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Actividad en Equipos - Semana 10 : LLM y IA en tu lugar de trabajo**

* **Nombres y matrículas:**

  *   Carlos Pano Hernandez - A01066264
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo: 64**


# **Parte 1: Carga de credenciales OpenAI**

In [ ]:
# Load environment variables from the specific .env file
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv('.env')

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = OpenAI(api_key=api_key)

# Prueba de la API de OpenAI
response = client.responses.create(
    model="gpt-4o-mini",
    input="Hola, como estas? (Responde en 1 palabra y en Aleman)"
)

# Imprimir la respuesta como test
print(response.output_text)

# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Hola! Stori es una Fintech que se dedica a la creación de productos financieros (tarjetas de crédito, cuentas de ahorro con dinero creciente, préstamos, etc.).
    En Github, tenemos una organización donde se encuentran TODOS los repositorios que contienen el código de Stori. Por lo que para este proyecto, la fuente será Github. Los repos de Github, son privados (token: api_key = os.getenv('CPANO_GITHUB_TOKEN'))
    
    Realiza el plantamiento de un nuevo proyecto de IA para Stori que resuelva el siguiente problema:
    - Problema: Cada repo, ejemplo (https://github.com/credifranco/deposits-transactions-securedcard/blob/dev/TROUBLESHOOTING_RUNBOOK.md), tiene un runbook que es un documento que contiene documentación relevante para dar soporte a ingenieros que no tengan contexto de como resolver un problema.
    - Solución: Realiza un pipeline de IA que lea el runbook y genere un resumen de lo que se menciona en el runbook. El runbook debe ser descargado de Github, leído e interpretado por un modelo de LLM para ser inyecto como un RAG. Usando Gradio, despliega una interfaz donde se puede ingresar una pregunta y el modelo de LLM genere una respuesta basada en el RAG de los runbooks. Pueden ser varios, entonces se debe diseñar un Pipeline de IA que lea los runbooks y genere un RAG para cada uno de ellos.
    - Resultados: Una interfaz que pueda resolver dudas en tiempo real de los ingenieros de Stori.
    
    Retorna el plantamiento de este proyecto, considerando las herramientas disponibles, RAG, LLM, etc. Así como el plan de acción para llevarlo a cabo.
    
    """
)

# Imprimir la respuesta como test
print(response.output_text)

In [ ]:
!pip install --user PyGithub TextLoader langchain_community langchain_openai chromadb gradio

In [ ]:
pip install --upgrade gradio

In [ ]:
# stori_runbook_chatbot_poc.ipynb

# --- Importar Librerías ---
import os
import shutil
import logging # Importa el módulo logging
from github import Github, UnknownObjectException
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from IPython.display import Markdown, display # Solo si estás ejecutando en Jupyter/IPython
import gradio as gr

# Cargar variables de entorno
from dotenv import load_dotenv
load_dotenv('.env')

# --- Configuración de Logging ---
# Configura el nivel de logging a INFO para ver los mensajes de depuración.
# Puedes cambiar a logging.DEBUG para mensajes más detallados si los añades.
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- 0. Configuración de Variables de Entorno y API Keys ---
logging.info("--- 0. Configuración de Variables de Entorno y API Keys ---")
GITHUB_TOKEN = os.getenv('CPANO_GITHUB_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not GITHUB_TOKEN:
    logging.error("Error: CPANO_GITHUB_TOKEN no está configurado. Por favor, configúralo en tus variables de entorno o en un archivo .env.")
    raise ValueError("Error: CPANO_GITHUB_TOKEN no está configurado. Por favor, configúralo en tus variables de entorno o en un archivo .env.")
if not OPENAI_API_KEY:
    logging.error("Error: OPENAI_API_KEY no está configurado. Por favor, configúralo en tus variables de entorno o en un archivo .env.")
    raise ValueError("Error: OPENAI_API_KEY no está configurado. Por favor, configúralo en tus variables de entorno o en un archivo .env.")
logging.info("Variables de entorno GITHUB_TOKEN y OPENAI_API_KEY cargadas.")

# --- Parte 1: Descarga del Runbook ---
logging.info("--- Parte 1: Descarga del Runbook de GitHub ---")

ORG_NAME = "credifranco"
REPO_NAME = "deposits-transactions-securedcard"
RUNBOOK_PATH = "TROUBLESHOOTING_RUNBOOK.md"

LOCAL_RUNBOOK_DIR = "./downloaded_runbooks"
LOCAL_RUNBOOK_FILE = os.path.join(LOCAL_RUNBOOK_DIR, f"{REPO_NAME}_{RUNBOOK_PATH.replace('/', '_')}")

os.makedirs(LOCAL_RUNBOOK_DIR, exist_ok=True)
download_successful = False
logging.info(f"Directorio local para runbooks: {LOCAL_RUNBOOK_DIR}")

try:
    g = Github(GITHUB_TOKEN)
    org = g.get_organization(ORG_NAME)
    repo = org.get_repo(REPO_NAME)
    logging.info(f"Conectado a GitHub. Org: {ORG_NAME}, Repo: {REPO_NAME}")
    
    file_content = None
    try:
        logging.info(f"Buscando el runbook en la rama 'dev'...")
        file_content = repo.get_contents(RUNBOOK_PATH, ref="dev")
        logging.info(f"✅ Runbook encontrado en la rama 'dev'.")
    except UnknownObjectException:
        logging.warning(f"❌ Runbook no encontrado en la rama 'dev'.")
        logging.info(f"Buscando en la rama por defecto del repositorio...")
        try:
            file_content = repo.get_contents(RUNBOOK_PATH)
            logging.info(f"✅ Runbook encontrado en la rama por defecto.")
        except UnknownObjectException:
            logging.error(f"❌ Runbook '{RUNBOOK_PATH}' NO encontrado en el repositorio '{ORG_NAME}/{REPO_NAME}' en ninguna de las ramas principales ('dev' o por defecto).")
            logging.error("Por favor, verifica la ruta del archivo y los permisos de tu token.")
    
    if file_content:
        decoded_content = file_content.decoded_content.decode('utf-8')
        with open(LOCAL_RUNBOOK_FILE, "w", encoding="utf-8") as f:
            f.write(decoded_content)
        logging.info(f"--- 🎉 ÉXITO EN LA DESCARGA 🎉 ---")
        logging.info(f"El runbook '{RUNBOOK_PATH}' se ha descargado exitosamente a: '{LOCAL_RUNBOOK_FILE}'")
        download_successful = True
    else:
        logging.warning("\n--- ⚠️ FALLO EN LA DESCARGA ⚠️ ---")
        logging.warning("No se pudo descargar el runbook. Verifica los mensajes anteriores.")

except Exception as e:
    logging.critical(f"--- ❌ ERROR CRÍTICO DURANTE LA DESCARGA ❌ ---")
    logging.critical(f"Ocurrió un error inesperado: {e}", exc_info=True) # exc_info=True para el traceback completo
    logging.critical("Asegúrate de que tu CPANO_GITHUB_TOKEN sea válido y tenga los permisos necesarios (repo) para acceder a la organización y el repositorio especificados.")

finally:
    if not download_successful:
        if os.path.exists(LOCAL_RUNBOOK_DIR) and not os.listdir(LOCAL_RUNBOOK_DIR):
            os.rmdir(LOCAL_RUNBOOK_DIR)
            logging.info(f"Directorio vacío '{LOCAL_RUNBOOK_DIR}' eliminado.")
        logging.info("--- No se puede continuar con la Parte 2 sin un runbook descargado. ---")
        exit() 

logging.info("\n--- Fin de la Parte 1 ---\n")

# --- Parte 2: Procesamiento del Runbook y Configuración del RAG ---
logging.info("--- Parte 2: Procesamiento del Runbook y Configuración del RAG ---")

# 2.1 Carga del Documento
logging.info("2.1 Cargando el runbook descargado...")
loader = TextLoader(LOCAL_RUNBOOK_FILE, encoding="utf-8")
documents = loader.load()
logging.info(f"Runbook cargado. Contiene {len(documents)} documentos (esperado: 1). Longitud del contenido: {len(documents[0].page_content)} caracteres.")

# 2.2 División del Documento en Chunks
logging.info("2.2 Dividiendo el documento en chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(documents)
logging.info(f"Documento dividido en {len(texts)} chunks.")
# logging.debug(f"Primer chunk: {texts[0].page_content[:200]}...") # Para depuración muy detallada

# 2.3 Generación de Embeddings y Almacenamiento en ChromaDB
logging.info("2.3 Generando embeddings y almacenando en ChromaDB...")
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
persist_directory = "./chroma_db"

# Limpiar ChromaDB si ya existe para asegurar una carga fresca
if os.path.exists(persist_directory):
    logging.info(f"Eliminando directorio existente de ChromaDB: {persist_directory}")
    shutil.rmtree(persist_directory)
    
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings_model,
    persist_directory=persist_directory
)
vectordb.persist()
logging.info(f"Embeddings generados y almacenados en ChromaDB en '{persist_directory}'.")

# 2.4 Configuración del LLM y la Cadena de RAG
logging.info("2.4 Configurando el LLM y la cadena de RAG...")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    openai_api_key=OPENAI_API_KEY
)
logging.info(f"LLM configurado: {llm.model_name}, temperatura: {llm.temperature}")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)
logging.info("LLM y cadena de RAG configurados exitosamente.")

logging.info("\n--- Fin de la Parte 2 ---\n")

# --- Parte 3: Interfaz de Chatbot con Gradio ---
# ... (all your imports and setup from above) ...

logging.info("--- Parte 3: Iniciando Interfaz de Chatbot con Gradio ---")

def chat_with_runbook(question, history):
    logging.info(f"Pregunta recibida del chatbot: '{question}'")

    if not question.strip():
        logging.warning("Pregunta vacía recibida.")
        # Return an error message and the current history *unchanged* if the input is invalid
        return "Por favor, ingresa una pregunta válida.", history

    try:
        logging.info(f"Invocando qa_chain con query: '{question}'")
        result = qa_chain.invoke({"query": question})
        logging.info(f"Resultado de qa_chain.invoke: {result}")

        if not isinstance(result, dict):
            raise TypeError(f"qa_chain.invoke() debe retornar un diccionario, pero retornó: {type(result)}")
        if "result" not in result:
            raise KeyError("La clave 'result' no se encontró en el diccionario de respuesta de qa_chain.")
        if "source_documents" not in result:
            raise KeyError("La clave 'source_documents' no se encontró en el diccionario de respuesta de qa_chain.")

        answer = result["result"]
        source_docs = result["source_documents"]

        logging.info(f"Respuesta generada (primeros 100 chars): {answer[:100]}...")
        logging.info(f"Número de documentos fuente recuperados: {len(source_docs)}")

        sources_info = "\n\n**Fuentes (Extractos):**\n"
        if source_docs:
            for i, doc in enumerate(source_docs):
                logging.info(f"Procesando documento fuente {i+1}. Tipo: {type(doc)}")
                if not hasattr(doc, 'page_content') or not isinstance(doc.page_content, str):
                    logging.error(f"Documento fuente {i+1} no tiene page_content válido. Saltando.")
                    continue

                page_content_preview = doc.page_content[:250].replace('\n', ' ')
                if len(doc.page_content) > 250:
                    page_content_preview += '...'
                sources_info += f"- **Chunk {i+1}:** {page_content_preview}\n"
                if hasattr(doc, 'metadata') and 'source' in doc.metadata:
                    sources_info += f" (Fuente: {doc.metadata['source']})\n"
                logging.debug(f"Chunk {i+1} añadido a fuentes: {page_content_preview}")
        else:
            sources_info = "\n\n*No se encontraron fuentes directas en los runbooks para esta consulta.*"
            logging.info("No se recuperaron documentos fuente para esta consulta.")

        full_response = f"{answer}{sources_info}"
        logging.info(f"Respuesta completa para Gradio (primeros 200 chars): {full_response[:200]}...")

        # IMPORTANT: Append the current turn (question, full_response) to the history
        # Gradio expects the *updated* history back.
        # Ensure 'history' is a list and append a list/tuple for the current turn.
        # If history is None or not initialized, ensure it starts as an empty list.
        if history is None:
            history = []
        history.append([question, full_response]) # Append the user's question and the bot's response

        return "", history # Return an empty string for the immediate output, and the updated history
        # Gradio's ChatInterface will then render the history.

    except Exception as e:
        error_message = f"Ocurrió un error al procesar tu pregunta: {e}"
        logging.error(f"Error durante el procesamiento de la pregunta: {e}", exc_info=True)
        # For errors, you still need to return a message and the history.
        # Here, we append the user's question and the error message to history.
        if history is None:
            history = []
        history.append([question, error_message])
        return "", history # Return an empty string for the output, and the updated history with the error.

# Crea la interfaz de Gradio para el chatbot
chatbot_interface = gr.ChatInterface(
    fn=chat_with_runbook,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Haz tu pregunta sobre el runbook...", container=False, scale=7),
    title="🤖 Asistente de Soporte de Stori (PoC RAG con Chatbot)",
    description="Pregunta sobre la documentación interna de los runbooks de troubleshooting de Stori. Las respuestas son generadas por un LLM basado en la información recuperada de tus documentos.",
    theme="soft"
)

# Lanza la interfaz de Gradio
logging.info("Gradio iniciado. Accede a la interfaz a través del enlace que aparecerá a continuación.")
chatbot_interface.launch(inbrowser=True, share=True)